# A Demonstration of Python's Power

Here's just a quick demonstration of how to accomplish a pretty interesting task, plotting a satellite image from a remote server, in Python. We won't explain much of what's going on here, but just want to show how much you can accomplish in Python.

First we just bring in some tools from a variety of Python libraries, using the `import` command. Python is really powerful at assembling various tools together like lego bricks.

In [ ]:
# A whole bunch of imports
%matplotlib inline
from urllib.request import urlopen

import matplotlib.pyplot as plt
from matplotlib import patheffects
import cartopy.crs as ccrs
import cartopy.feature as cfeat

from metpy.plots import colortables

from siphon.catalog import TDSCatalog

from xarray import open_dataset

Now we download the latest water vapor satellite imagery from a remote server and pull out a variety of needed information. This makes use Unidata's MetPy and Siphon libraries.

In [ ]:
# Scan the catalog and download the data

cat = TDSCatalog(
    'https://thredds.ucar.edu/thredds/catalog/satellite/goes/east/products/'
    'CloudAndMoistureImagery/CONUS/Channel09/current/catalog.xml'
)
dataset = cat.datasets[list(cat.datasets)[0]]
#gini_file = GiniFile(urlopen(dataset.access_urls['HTTPServer']))
sat_ds = open_dataset(dataset.access_urls['OPENDAP']).metpy.parse_cf()
# Pull parts out of the data file
data_var = sat_ds.Sectorized_CMI
x = sat_ds.x.values
y = sat_ds.y.values
time_var = sat_ds.time.values.astype('datetime64[ms]').astype('O')
proj_var = sat_ds.fixedgrid_projection

# Set up the projection
proj = ccrs.Geostationary(
    central_longitude=proj_var.attrs['longitude_of_projection_origin'],
    satellite_height=proj_var.attrs['perspective_point_height'])

Now we plot the satellite data along with some mapping information; this plotting is provided by the matplotlib library, while the mapping capabilities come from a library called cartopy.

In [ ]:
# Create the figure
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=proj)

wv_norm, wv_cmap = colortables.get_with_range('WVCIMSS_r', 164, 294)
wv_cmap.set_under('k')
im = ax.imshow(
    data_var, cmap=wv_cmap, norm=wv_norm,
    extent=(x.min(), x.max(), y.min(), y.max()),
    origin='upper')
plt.colorbar(im, orientation='horizontal', pad=0, aspect=50)

# Add mapping information
ax.add_feature(cfeat.COASTLINE.with_scale('50m'), edgecolor='black')
ax.add_feature(cfeat.STATES, edgecolor='red', linestyle=':')
ax.add_feature(cfeat.BORDERS, linewidth=2, edgecolor='black')

text = ax.text(
    0.99, 0.01, time_var, horizontalalignment='right',
    transform=ax.transAxes, color='white', fontsize='x-large', weight='bold')

text.set_path_effects([
    patheffects.Stroke(linewidth=2, foreground='black'),
    patheffects.Normal()
])